<a href="https://colab.research.google.com/github/wuba2010work/CWP/blob/main/NPP_estimation_CASA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import xarray as xr

def calculate_fPAR(NDVI, NDVI_min, NDVI_max, fPAR_max, fPAR_min):
    """
    Calculates fPAR using the provided equation.

    Args:
        NDVI: A NumPy array or a single value representing the NDVI.
        NDVI_min: The minimum NDVI value.
        NDVI_max: The maximum NDVI value.
        fPAR_max: The maximum fPAR value.
        fPAR_min: The minimum fPAR value.

    Returns:
        A NumPy array or a single value representing the calculated fPAR.
    """

    # Ensure inputs are compatible for array operations
    NDVI = np.asarray(NDVI)  # Convert NDVI to a NumPy array if it's not already

    # Handle the case where NDVI_max - NDVI_min is zero to avoid division by zero
    denominator = NDVI_max - NDVI_min
    if denominator == 0:
        # If NDVI_max and NDVI_min are equal, NDVI will always be the same, so FPAR will be constant
        fPAR = np.full_like(NDVI, fPAR_min) # Or fPAR_max, it doesn't matter since NDVI is constant
    else:
        fPAR = (NDVI - NDVI_min) * (fPAR_max - fPAR_min) / denominator + fPAR_min

    return fPAR

In [ ]:
# calculate LUE parametres



def compute_T_e1(T_opt):
    """
    Computes the temperature stress factor T_e1 based on the optimal temperature.

    Parameters:
    T_opt : numpy array
        Optimal temperature for each pixel (x) and time (t).

    Returns:
    T_e1 : numpy array
        Temperature stress factor 1 for each pixel (x) and time (t).
    """
    return 0.8 + 0.002 * T_opt - 0.0005 * (T_opt ** 2)

def compute_T_e2(T_opt, T_actual):
    """
    Computes the temperature stress factor T_e2 based on the optimal and actual temperatures.

    Parameters:
    T_opt : numpy array
        Optimal temperature for each pixel (x) and time (t).
    T_actual : numpy array
        Actual temperature for each pixel (x) and time (t).

    Returns:
    T_e2 : numpy array
        Temperature stress factor 2 for each pixel (x) and time (t).
    """
    term1 = 1 + np.exp(0.2 * (T_opt - 10 - T_actual))
    term2 = 1 + np.exp(0.3 * (-T_opt - 10 + T_actual))
    return 1.184 / (term1 * term2)

def compute_W_e(EET, PET):
    """
    Computes the water stress factor W_e based on evapotranspiration values.

    Parameters:
    EET : numpy array
        Actual evapotranspiration for each pixel (x) and time (t).
    PET : numpy array
        Potential evapotranspiration for each pixel (x) and time (t).

    Returns:
    W_e : numpy array
        Water stress factor for each pixel (x) and time (t).
    """
    return (0.5 * (EET / PET)) + 0.5

def compute_lue(T_e1, T_e2, W_e, epsilon_star):
    """
    Computes the actual Light Use Efficiency (LUE) based on temperature and water stress factors.

    Parameters:
    T_e1 : numpy array
        Temperature stress factor 1 for each pixel (x) and time (t).
    T_e2 : numpy array
        Temperature stress factor 2 for each pixel (x) and time (t).
    W_e : numpy array
        Water stress factor for each pixel (x) and time (t).
    epsilon_star : float
        Maximum light use efficiency under ideal conditions [g C MJ^-1].

    Returns:
    epsilon : numpy array
        Actual light use efficiency for each pixel (x) and time (t).
    """
    epsilon = T_e1 * T_e2 * W_e * epsilon_star
    return epsilon

In [ ]:
# ETa

def calculate_EET(P, Rn):
    """Calculates Actual Evapotranspiration (EET) using Equation 16.

    Args:
        P: NumPy array or single value for precipitation (mm).
        Rn: NumPy array or single value for net radiation (MJ m-2).

    Returns:
        NumPy array or single value for calculated EET (mm).
        Returns NaN if any input is NaN.
        Raises ValueError if input shapes are incompatible for calculations.
    """

    P = np.asarray(P)
    Rn = np.asarray(Rn)

    # Handle NaN values
    if np.isnan(P).any() or np.isnan(Rn).any():
        return np.full_like(P, np.nan, dtype=float)

    # Check for shape compatibility (broadcasting allowed)
    if P.shape != Rn.shape and P.ndim > 0 and Rn.ndim > 0 and not (P.shape == () or Rn.shape == ()):
        raise ValueError("P and Rn must have compatible shapes for calculations (broadcasting allowed).")


    numerator = P * Rn * (P**2 + Rn**2 + P * Rn)
    denominator = (P + Rn) * (P**2 + Rn**2)

    # Avoid division by zero
    denominator[denominator == 0] = np.nan  # Set 0 values in denominator to NaN

    EET = numerator / denominator

    return EET

In [ ]:
# # calculate Rn

def calculate_Rn(Epo, P):
    """Calculates Net Radiation (Rn) using Equation 17.

    Args:
        Epo: NumPy array or single value for local potential evapotranspiration.
        P: NumPy array or single value for precipitation (mm).

    Returns:
        NumPy array or single value for calculated Rn (MJ m-2).
        Returns NaN if any input is NaN or if P is 0.
        Raises ValueError if input shapes are incompatible for calculations.
    """

    Epo = np.asarray(Epo)
    P = np.asarray(P)

    # Handle NaN values
    if np.isnan(Epo).any() or np.isnan(P).any():
        return np.full_like(Epo, np.nan, dtype=float)

     # Check for shape compatibility (broadcasting allowed)
    if Epo.shape != P.shape and Epo.ndim > 0 and P.ndim > 0 and not (Epo.shape == () or P.shape == ()):
        raise ValueError("Epo and P must have compatible shapes for calculations (broadcasting allowed).")

    # Handle P == 0 to avoid division by zero and math domain error
    with np.errstate(divide='ignore', invalid='ignore'): # Suppress warnings for division by zero and invalid sqrt
        sqrt_term = np.sqrt(Epo / P)
        sqrt_term[P == 0] = np.nan # Set results where P==0 to NaN

    Rn = np.sqrt(Epo * P) * (0.369 + 0.598 * sqrt_term)

    return Rn

In [ ]:
# calculate PET

def calculate_PET(Epo, EET):
    """Calculates Potential Evapotranspiration (PET) using Equation 18.

    Args:
        Epo: NumPy array or single value for local potential evapotranspiration.
        EET: NumPy array or single value for actual evapotranspiration.

    Returns:
        NumPy array or single value for calculated PET (mm).
        Returns NaN if any input is NaN.
        Raises ValueError if input shapes are incompatible for calculations.
    """
    Epo = np.asarray(Epo)
    EET = np.asarray(EET)

    # Handle NaN values
    if np.isnan(Epo).any() or np.isnan(EET).any():
        return np.full_like(Epo, np.nan, dtype=float)

    # Check for shape compatibility (broadcasting allowed)
    if Epo.shape != EET.shape and Epo.ndim > 0 and EET.ndim > 0 and not (Epo.shape == () or EET.shape == ()):
        raise ValueError("Epo and EET must have compatible shapes for calculations (broadcasting allowed).")

    PET = (Epo + EET) / 2

    return PET

In [ ]:
# calculate NPP

def calculate_NPP(APAR, epsilon):
    """
    Calculates Net Primary Productivity (NPP) using the formula: NPP = APAR * epsilon

    Args:
        APAR: A NumPy array or a single value representing Absorbed Photosynthetically Active Radiation.
        epsilon: A NumPy array or a single value representing light use efficiency.

    Returns:
        A NumPy array or a single value representing the calculated NPP.  Returns NaN if either input is NaN.
        Raises a ValueError if the shapes of APAR and epsilon are incompatible for element-wise multiplication.
    """

    APAR = np.asarray(APAR)  # Ensure APAR is a NumPy array
    epsilon = np.asarray(epsilon) # Ensure epsilon is a NumPy array

    # Handle NaN values: If either input is NaN, the output should also be NaN
    if np.isnan(APAR).any() or np.isnan(epsilon).any():
       return np.full_like(APAR, np.nan, dtype=float) # Return an array of NaNs of the same shape

    # Check for shape compatibility for element-wise multiplication
    if APAR.shape != epsilon.shape and APAR.ndim > 0 and epsilon.ndim > 0 and not (APAR.shape == () or epsilon.shape == ()): # Check if both are arrays with at least one dimension and not a scalar
        raise ValueError("APAR and epsilon must have compatible shapes for element-wise multiplication or one of them needs to be a scalar.")

    NPP = APAR * epsilon  # Perform the calculation

    return NPP

